# Performance measurements for scheduler run

## System information

In [5]:
%%python

import os
print (os.popen("lscpu").read())

Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Address sizes:                        48 bits physical, 48 bits virtual
Byte Order:                           Little Endian
CPU(s):                               12
On-line CPU(s) list:                  0-11
Vendor ID:                            AuthenticAMD
Model name:                           AMD Ryzen 5 7600X 6-Core Processor
CPU family:                           25
Model:                                97
Thread(s) per core:                   2
Core(s) per socket:                   6
Socket(s):                            1
Stepping:                             2
BogoMIPS:                             9382.23
Flags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl tsc_reliable nonstop_tsc cpuid extd_apicid pni pclmulqdq ssse3 fma cx16

In [6]:
%%python
print (os.popen("nvidia-smi").read())

Wed Jun 11 14:08:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.55.01              Driver Version: 576.40         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070        On  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P5             12W /  250W |    5897MiB /  12227MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Test run

In [7]:
%%python
# Adjust the path to your build directory
buildDir="../../build/wp1.7-scheduler-test"
print (os.popen(buildDir + "/bin/simple_scheduler_scan.sh | tee " + buildDir + "/simple_scheduler_scan.csv").read())

Slots,Threads,Bandwidth
5,1,3521.13
10,1,4040.4
15,1,5102.04
20,1,5464.48
25,1,4761.9
30,1,5665.72
35,1,5167.96
40,1,5305.04
45,1,5154.64
50,1,5221.93
5,2,4282.66
10,2,4914
15,2,4716.98
20,2,4694.84
25,2,4878.05
30,2,4926.11
35,2,4842.62
40,2,5405.41
45,2,5000
50,2,4514.67
5,3,4319.65
10,3,4454.34
15,3,4246.28
20,3,4098.36
25,3,4662
30,3,4106.78
35,3,4618.94
40,3,3984.06
45,3,4395.6
50,3,4454.34
5,4,4395.6
10,4,4366.81
15,4,3759.4
20,4,4405.29
25,4,4329
30,4,4310.34
35,4,4319.65
40,4,4040.4
45,4,4192.87
50,4,4597.7
5,5,4357.3
10,5,4210.53
15,5,4454.34
20,5,4385.96
25,5,4385.96
30,5,4484.3
35,5,4291.85
40,5,3960.4
45,5,4357.3
50,5,4395.6
5,6,4140.79
10,6,4514.67
15,6,4376.37
20,6,4310.34
25,6,4291.85
30,6,4237.29
35,6,4210.53
40,6,4329
45,6,4338.39
50,6,4192.87
5,7,4000
10,7,4301.08
15,7,4073.32
20,7,4192.87
25,7,4056.8
30,7,4081.63
35,7,3717.47
40,7,4166.67
45,7,4219.41
50,7,4123.71
5,8,3898.64
10,8,4132.23
15,8,4158
20,8,4149.38
25,8,4140.79
30,8,3891.05
35,8,4273.5
40,8,4237.29
45,8,

In [12]:
#include <ROOT/RDataFrame.hxx>
#include <numeric>

// Adjust the path to your CSV file
std::string buildDir = "../../build/wp1.7-scheduler-test/";
std::string csvfile = buildDir + "simple_scheduler_scan.csv";

// Load CSV with columns: Slots,Threads,Rate
auto df = ROOT::RDF::FromCSV(csvfile);

std::cout << df.Count().GetValue() << " rows loaded from " << csvfile << std::endl;
std::cout << "Columns: ";
for (auto & c: df.GetColumnNames())
  std::cout << c << " ";
std::cout << std::endl;

auto cRate = new TCanvas("cRate", "Rate vs Threads and Slots", 1600, 800);
cRate->Divide(2, 1);
auto gflat = new TGraph2D();
auto g3d = new TGraph2D();
gflat->SetTitle("Rate vs Threads and Slots;Slots;Threads;Rate (evts/s)");
g3d->SetTitle("Rate vs Threads and Slots;Slots;Threads;Rate (evts/s)");

int i=0;
df.Foreach([&](double b, Long64_t s, Long64_t t){gflat->SetPoint(i, s, t, b);g3d->SetPoint(i++, s, t, b);}, {"Rate", "Slots", "Threads"});
cRate->cd(1);
gflat->Draw("COLZ RY");
cRate->cd(2);
g3d->Draw("SURF1 RY");
gStyle->SetPalette(1);
cRate->Draw();

100 rows loaded from ../../build/wp1.7-scheduler-test/simple_scheduler_scan.csv
Columns: Rate Slots Threads 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cRate
